<a href="https://colab.research.google.com/github/fnitiwat/colab/blob/main/knowledge_sharing_feast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README
- setup
- raw data
- transformed data
- feature store config
- featore store repo
- feast apply
- get historical feature
- materialize
- get online feature
- tear down

# Setup

In [1]:
! git clone https://github.com/fnitiwat/movielen_recsys.git

Cloning into 'movielen_recsys'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 125 (delta 0), reused 0 (delta 0), pack-reused 120
Receiving objects: 100% (125/125), 41.66 MiB | 22.29 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [2]:
! apt-get install tree
! pip install feast

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 17 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (480 kB/s)
Selecting previously unselected package tree.
(Reading database ... 120901 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6

In [3]:
import os
os.chdir('/content/movielen_recsys/feature_store/src')

# Raw Data
- ratings


In [4]:
! tree ./data

./data
├── interim
│   ├── transformed_global_stats.csv
│   ├── transformed_global_stats.parquet
│   ├── transformed_ratings.csv
│   ├── transformed_ratings.parquet
│   ├── transformed_watch_histories.csv
│   └── transformed_watch_histories.parquet
└── raw
    ├── LICENSE
    ├── links.csv
    ├── movies.csv
    ├── ratings.csv
    ├── README.md
    └── tags.csv

2 directories, 12 files


In [5]:
import pandas as pd

df = pd.read_csv("./data/raw/ratings.csv")
display(df)

,userId,movieId,rating,timestamp
0,1.0,1.0,4.0,964982703
1,1.0,3.0,4.0,964981247
2,1.0,6.0,4.0,964982224
3,1.0,47.0,5.0,964983815
4,1.0,50.0,5.0,964982931
...,...,...,...,...
100831,610.0,166534.0,4.0,1493848402
100832,610.0,168248.0,5.0,1493850091
100833,610.0,168250.0,5.0,1494273047
100834,610.0,168252.0,5.0,1493846352


# Transformed Data
- transformed_rating
  - update timestamp format
- watch_histories
  - transform rating to array of watch history in each time stamp

In [6]:
df = pd.read_csv("./data/interim/transformed_ratings.csv")
display(df)

,Unnamed: 0,userId,movieId,rating,timestamp
0,0,1.0,1.0,4.0,2000-07-31 01:45:03
1,1,1.0,3.0,4.0,2000-07-31 01:20:47
2,2,1.0,6.0,4.0,2000-07-31 01:37:04
3,3,1.0,47.0,5.0,2000-07-31 02:03:35
4,4,1.0,50.0,5.0,2000-07-31 01:48:51
...,...,...,...,...,...
100831,100831,610.0,166534.0,4.0,2017-05-04 04:53:22
100832,100832,610.0,168248.0,5.0,2017-05-04 05:21:31
100833,100833,610.0,168250.0,5.0,2017-05-09 02:50:47
100834,100834,610.0,168252.0,5.0,2017-05-04 04:19:12


In [7]:
df = pd.read_csv("./data/interim/transformed_watch_histories.csv")
display(df)

,Unnamed: 0,userId,watch_histories,timestamp
0,0,1.0,[1.0],2000-07-31 01:45:03
1,1,1.0,"[3.0, 1.0]",2000-07-31 01:20:47
2,2,1.0,"[6.0, 3.0, 1.0]",2000-07-31 01:37:04
3,3,1.0,"[47.0, 6.0, 3.0, 1.0]",2000-07-31 02:03:35
4,4,1.0,"[50.0, 47.0, 6.0, 3.0, 1.0]",2000-07-31 01:48:51
...,...,...,...,...
100831,80068,NaN,"[3977.0, 3671.0, 3623.0, 3471.0, 3469.0, 3147....",2005-03-25 20:28:41
100832,80069,NaN,"[4034.0, 3977.0, 3671.0, 3623.0, 3471.0, 3469....",2005-03-25 20:27:36
100833,80070,NaN,"[4412.0, 4034.0, 3977.0, 3671.0, 3623.0, 3471....",2005-03-25 20:35:12
100834,80071,NaN,"[7236.0, 4412.0, 4034.0, 3977.0, 3671.0, 3623....",2005-03-25 20:34:26


# Feature Store Config

In [8]:
! cat ./feature_store.yaml

registry: data/registry.db
project: movielen_recsys
provider: local
flags:
  alpha_features: true
  on_demand_transforms: true
online_store:
  path: data/online_store.db
  type: sqlite
offline_store:
  type: file
_offline_store: null
_offline_config: file
_online_store: null
_online_config:
  path: data/online_store.db
  type: sqlite


# Feature Store Repo

In [9]:
! cat feature_repo.py

import pandas as pd
import numpy as np
from feast import (
    FileSource,
    Entity,
    FeatureView,
    FeatureService,
    Field,
    types,
    ValueType,
)
from feast.on_demand_feature_view import on_demand_feature_view
from feast.types import Int64


################################
# rating
################################
# TODO: make rating correct (support 2 key userId, movieId)
rating_entity = Entity(name="rating_entity", join_keys=["userId"])
rating_feature_view = FeatureView(
    name="rating_feature_view",
    entities=[rating_entity],
    source=FileSource(
        name="rating_source",
        path="./data/interim/transformed_ratings.parquet",
        timestamp_field="timestamp",
    ),
    ttl=None,
    online=True,
    tags={},
)


################################
# watch histories
################################
watch_histories_entity = Entity(name="watch_histories_entity", join_keys=["userId"])
watch_histories_feature_view = FeatureView(
    name="watch_histories

# Apply

In [10]:
! feast apply

Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage
09/17/2023 09:34:25 PM feast.repo_config WARNING: Flags are no longer necessary in Feast. Experimental features will log warnings instead.
/usr/local/lib/python3.10/dist-packages/feast/repo_config.py:233: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
Created entity watch_histories_entity
Created entity rating_entity
Created feature view global_stats_view
Created feature view rating_feature_view
Created feature view watch_histories_feature_view
Created feature service model_v1
Created feature service model_v3
Created feat

# Retrieval



In [11]:
import feast
import pandas as pd
from datetime import datetime

fs = feast.FeatureStore(repo_path=".")

/usr/local/lib/python3.10/dist-packages/feast/repo_config.py:233: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(


### Data Scientist Research Retrieval

In [12]:
entity_df = pd.DataFrame({"userId": ["1.0", "2.0", "3.0", "4.0", "5.0"], "event_timestamp": [datetime.now()]*5})
training_df = fs.get_historical_features(
    entity_df=entity_df,
    features=[
        "watch_histories_feature_view:watch_histories",
        "rating_feature_view:movieId",
        "rating_feature_view:rating",
        "global_stats_view:movieIds",
    ],
).to_df()
display(training_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,event_timestamp,watch_histories,movieId,rating,movieIds
0,3.0,2023-09-17 21:34:27.737541+00:00,NaN,NaN,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,5.0,2023-09-17 21:34:27.737541+00:00,"[300.0, 296.0, 290.0, 266.0, 265.0, 261.0, 253...",247.0,5.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,1.0,2023-09-17 21:34:27.737541+00:00,"[2492.0, 2478.0, 2470.0, 2459.0, 2450.0, 2427....",2492.0,4.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,4.0,2023-09-17 21:34:27.737541+00:00,"[4246.0, 4239.0, 4226.0, 4166.0, 4144.0, 4121....",4246.0,4.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,2.0,2023-09-17 21:34:27.737541+00:00,"[80489.0, 79132.0, 77455.0, 74458.0, 71535.0, ...",80489.0,4.5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


### Training Pipeline Retrieval

In [13]:
entity_df = pd.DataFrame({"userId": ["1.0", "2.0", "3.0", "4.0", "5.0"], "timestamp": [datetime.now()]*5})
feature_service = fs.get_feature_service("rating_service")

training_df = fs.get_historical_features(
    features=feature_service, entity_df=entity_df
).to_df()
display(training_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Using timestamp as the event timestamp. To specify a column explicitly, please name it event_timestamp.


,userId,timestamp,movieId,rating
0,3.0,2023-09-17 21:34:29.502831+00:00,NaN,NaN
1,5.0,2023-09-17 21:34:29.502831+00:00,300.0,3.0
2,1.0,2023-09-17 21:34:29.502831+00:00,2492.0,4.0
3,4.0,2023-09-17 21:34:29.502831+00:00,4246.0,4.0
4,2.0,2023-09-17 21:34:29.502831+00:00,80489.0,4.5


# Materialize

In [14]:
! feast  materialize  1997-07-16T19:20:01 2024-07-16T19:20:01

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


09/17/2023 09:34:36 PM feast.repo_config WARNING: Flags are no longer necessary in Feast. Experimental features will log warnings instead.
/usr/local/lib/python3.10/dist-packages/feast/repo_config.py:233: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
Materializing 3 feature views from 1997-07-16 19:20:01+00:00 to 2024-07-16 19:20:01+00:00 into the sqlite online store.

global_stats_view:
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 29.19it/s]
rating_feature_view:
100%|███████████████████████████████████████████████████████████| 476/476 [00:00<00:00, 6974.44it/s]
watch_histories_feature_view:
100%|████████████████████████████████████████

### Online Serving Retrieval


In [15]:
entity_rows = [{"userId": "1.0"}, {"userId": "2.0"}]
feature_service = fs.get_feature_service("rating_service")

training_df = fs.get_online_features(
    features=feature_service, entity_rows=entity_rows
).to_df()
display(training_df)

,userId,movieId,rating
0,1.0,2492.0,4.0
1,2.0,80489.0,4.5


In [16]:
feature_service = fs.get_feature_service("watch_histories_service")

training_df = fs.get_online_features(
    features=feature_service, entity_rows=entity_rows
).to_df()
display(training_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,watch_histories
0,1.0,"[2492.0, 2478.0, 2470.0, 2459.0, 2450.0, 2427...."
1,2.0,"[80489.0, 79132.0, 77455.0, 74458.0, 71535.0, ..."


In [17]:
feature_service = fs.get_feature_service("model_v1")

training_df = fs.get_online_features(
    features=feature_service, entity_rows=entity_rows
).to_df()
display(training_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId,rating
0,1.0,2492.0,4.0
1,2.0,80489.0,4.5


In [18]:
feature_service = fs.get_feature_service("model_v2")

training_df = fs.get_online_features(
    features=feature_service, entity_rows=entity_rows
).to_df()
display(training_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,watch_histories
0,1.0,"[2492.0, 2478.0, 2470.0, 2459.0, 2450.0, 2427...."
1,2.0,"[80489.0, 79132.0, 77455.0, 74458.0, 71535.0, ..."


In [19]:
feature_service = fs.get_feature_service("model_v3")

training_df = fs.get_online_features(
    features=feature_service, entity_rows=entity_rows
).to_df()
display(training_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId,rating,watch_histories
0,1.0,2492.0,4.0,"[2492.0, 2478.0, 2470.0, 2459.0, 2450.0, 2427...."
1,2.0,80489.0,4.5,"[80489.0, 79132.0, 77455.0, 74458.0, 71535.0, ..."


# Tear Down

In [20]:
! feast teardown

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


09/17/2023 09:34:45 PM feast.repo_config WARNING: Flags are no longer necessary in Feast. Experimental features will log warnings instead.
/usr/local/lib/python3.10/dist-packages/feast/repo_config.py:233: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
09/17/2023 09:34:45 PM feast.repo_config WARNING: Flags are no longer necessary in Feast. Experimental features will log warnings instead.
